# Feedforward Network

Quelle: https://github.com/yunjey/pytorch-tutorial/blob/master/tutorials/01-basics/feedforward_neural_network/main.py#L37-L49

In [1]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

In [2]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
# Hyper-parameters 
input_size = 784
hidden_size = 500
num_classes = 10
num_epochs = 5
batch_size = 100
learning_rate = 0.001

In [4]:
# MNIST dataset 
train_dataset = torchvision.datasets.MNIST(root='../../data', 
                                           train=True, 
                                           transform=transforms.ToTensor(),  
                                           download=True)

test_dataset = torchvision.datasets.MNIST(root='../../data', 
                                          train=False, 
transform=transforms.ToTensor())

Processing...
Done!


In [5]:
# Data loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
                                          batch_size=batch_size, 
shuffle=False)

In [8]:
# Fully connected neural network with one hidden layer
class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size) 
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, num_classes)  
    
    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out

In [9]:
model = NeuralNet(input_size, hidden_size, num_classes).to(device)
model

NeuralNet(
  (fc1): Linear(in_features=784, out_features=500, bias=True)
  (relu): ReLU()
  (fc2): Linear(in_features=500, out_features=10, bias=True)
)

In [11]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [12]:
# Train the model
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):  
        # Move tensors to the configured device
        images = images.reshape(-1, 28*28).to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))


Epoch [1/5], Step [100/600], Loss: 0.5112
Epoch [1/5], Step [200/600], Loss: 0.4382
Epoch [1/5], Step [300/600], Loss: 0.1518
Epoch [1/5], Step [400/600], Loss: 0.1772
Epoch [1/5], Step [500/600], Loss: 0.1541
Epoch [1/5], Step [600/600], Loss: 0.1903
Epoch [2/5], Step [100/600], Loss: 0.1016
Epoch [2/5], Step [200/600], Loss: 0.0736
Epoch [2/5], Step [300/600], Loss: 0.0534
Epoch [2/5], Step [400/600], Loss: 0.1067
Epoch [2/5], Step [500/600], Loss: 0.1120
Epoch [2/5], Step [600/600], Loss: 0.1461
Epoch [3/5], Step [100/600], Loss: 0.0651
Epoch [3/5], Step [200/600], Loss: 0.0763
Epoch [3/5], Step [300/600], Loss: 0.1089
Epoch [3/5], Step [400/600], Loss: 0.0377
Epoch [3/5], Step [500/600], Loss: 0.0533
Epoch [3/5], Step [600/600], Loss: 0.1134
Epoch [4/5], Step [100/600], Loss: 0.0671
Epoch [4/5], Step [200/600], Loss: 0.0512
Epoch [4/5], Step [300/600], Loss: 0.0282
Epoch [4/5], Step [400/600], Loss: 0.0372
Epoch [4/5], Step [500/600], Loss: 0.0274
Epoch [4/5], Step [600/600], Loss:

In [13]:
# Test the model
# In test phase, we don't need to compute gradients (for memory efficiency)
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.reshape(-1, 28*28).to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: {} %'.format(100 * correct / total))

Accuracy of the network on the 10000 test images: 97.42 %


In [14]:
# Save the model checkpoint
torch.save(model.state_dict(), 'model.ckpt')